In [12]:
import subprocess
import os
import pickle
from datasets import load_dataset
from collections import Counter

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value
os.environ["HF_DATASETS_CACHE"] = "/root/autodl-tmp/cache/"

begin = 0
end = 100000

In [9]:
ds = load_dataset("wikimedia/wikipedia", "20231101.en")

KeyboardInterrupt: 

In [3]:
import re
 
def split_sentences(text):
    # 构建正则表达式，匹配任意标点符号或者连续多个空格
    pattern = r'[,.!\?\(\)]| {2,}'
    
    # 使用split函数根据正则表达式切分文本
    sentences = re.split(pattern, text)
    
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
    
    return sentences
 
# 测试样例
result = split_sentences(ds['train'][0]['text'])
print(len(result))

734


In [19]:
from nltk import ngrams
from nltk.tokenize import word_tokenize
from collections import Counter
from tqdm import tqdm
import time

def count_ngrams_in_sentences(sentences, n):
    all_ngram_counts = Counter()

    for sentence in sentences:
        words = word_tokenize(sentence)
        n_grams = ngrams(words, n)
        
        # 使用Counter统计当前句子的n-grams的数量
        ngram_counts = Counter(n_grams)

        # 将当前句子的n-grams数量添加到总的n-grams数量中
        all_ngram_counts += ngram_counts

    return all_ngram_counts


# sentences = list()
# for i in tqdm(range(begin, end)):
#     sentences.extend(split_sentences(ds['train'][i]['text']))
# four_gram_counts = count_ngrams_in_sentences(sentences, 4)

# print("所有句子的4-gram数量统计:")
# for four_gram, count in four_gram_counts.items():
#     if count > 5:
#         print(f"{four_gram}: {count}")
# ## 100句用了14分钟，tqdm可以看出来越跑越慢

['aa', 'bb', 'abc']


In [5]:
## 改为递归，14分钟变7秒！ 10万个花了26分钟
def count_ngrams(sentences, n):
    if len(sentences) <=  1:
        counts = count_ngrams_in_sentences(sentences, n)
        return counts
    else:
        half = len(sentences) // 2
        counts = count_ngrams(sentences[:half], n) + count_ngrams(sentences[half:], n)
        return counts

sentences = list()
for i in tqdm(range(begin, end)):
    sentences.extend(split_sentences(ds['train'][i]['text']))
    
start_time = time.time()

four_gram_counts = count_ngrams(sentences, 4)

end_time = time.time()
print(end_time-start_time)

# print("所有句子的4-gram数量统计:")
# for four_gram, count in four_gram_counts.items():
#     if count > 5:
#         print(f"{four_gram}: {count}")

100%|██████████| 100000/100000 [00:26<00:00, 3737.00it/s]


1569.715073108673


In [6]:
# 将 Counter 对象保存到本地文件
with open(f'4grams_{begin}_{end}.pkl', 'wb') as f:
    pickle.dump(four_gram_counts, f)


In [20]:
# 从本地文件加载 Counter 对象
with open('/root/autodl-tmp/4grams_0_100000.pkl', 'rb') as f:
    loaded_counter = pickle.load(f)
s = 0
for four_gram, count in loaded_counter.items():
    if count > 20:
        s += 1
print(s)

66527


In [8]:
print(len(four_gram_counts.items()))
print(len(loaded_counter.items()))

39611095
39611095
